In [63]:
from scipy.optimize import minimize_scalar
from scipy.optimize import fsolve
from scipy import optimize
import numpy as np
from pathlib import Path
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
import numpy as np
from pathlib import Path
import pandas as pd
from IPython.display import display, HTML
import matplotlib.pyplot as plt
from matplotlib.ticker import AutoMinorLocator


datapath = Path.cwd().parent / "data" / "eval" / "eval_cls_generalizability.csv"
data = pd.read_csv(datapath)


"""
preprocessing
"""


def get_quality(cs, psnr, ssim, lpips):
    # weighted average metric:
    # - Cosine Similarity: already in the range [-1, 1], can be used as is.
    # - PSNR: normalize to [0, 1] by using a formula like: (PSNR - 30) / 20, clamped to [0, 1].
    # - SSIM: already in the range [-1, 1], can be used as is.
    # - LPIPS: invert and normalize to [0, 1] using: 1 - LPIPS, clamped to [0, 1].

    # create a boolean mask for valid entries
    valid_mask = ~(np.isnan(cs) | np.isnan(psnr) | np.isnan(ssim) | np.isnan(lpips) |
                   np.isinf(cs) | np.isinf(psnr) | np.isinf(ssim) | np.isinf(lpips))
    
    # initialize with NaN
    wam = pd.Series(np.nan, index=cs.index)
    
    psnr_n = np.clip((psnr - 30) / 20, 0, 1)
    
    lpips_n = np.clip(lpips, 0, 1)
    
    wam[valid_mask] = (0.15 * cs[valid_mask] + 
                       0.25 * psnr_n[valid_mask] + 
                       0.35 * ssim[valid_mask] + 
                       0.25 * (1 - lpips_n[valid_mask]))
    
    return wam


data["acc_rank_delta"] = data["acc_rank"] - data["adv_acc_rank"] # before - after (lower is better)
data["quality"] = get_quality(data["cosine_sim"], data["psnr"], data["ssim"], data["lpips"])
data["final_score"] = data["quality"] - data["acc_rank_delta"] # higher is better

data = data.drop(columns=["model", "density", "img_id", "acc_rank", "adv_acc_rank", "cosine_sim", "psnr", "ssim", "lpips"])

data = data.groupby(['mask', 'opacity']).agg({
    'acc_rank_delta': 'mean',
    'quality': 'mean',
    'final_score': 'mean'
}).reset_index()
data = data.sort_values(by='final_score', ascending=False)

data

# """
# plot
# """


# plt.figure(figsize=(15, 15))

# adv_acc = data['acc_rank_delta'] # x-axis
# quality = data['quality'] # y-axis

# # invert order of x-axis
# plt.gca().invert_xaxis()

# model = data['model'] # "model": ["vit", "eva02", "eva01", "convnext", "resnet"],
# mask = data['mask'] # "mask": ["circle", "square", "diamond", "knit"],
# opacity = data['opacity'] # "opacity": [50, 80, 110, 140, 170],

# markers = {'circle': 'o', 'square': 's', 'diamond': 'D', 'knit': '^'}

# for mask in markers:
#     mask_data = data[data['mask'] == mask]
#     plt.scatter(mask_data['acc_rank_delta'], mask_data['quality'],
#                 marker=markers[mask],
#                 c='black',
#                 alpha=mask_data['opacity']/255,
#                 s=230,
#                 edgecolors='none')


# """
# polynomial regression
# """

# x = data['acc_rank_delta'].values.reshape(-1, 1)
# y = data['quality'].values

# total_degree = 1
# polyreg = make_pipeline(PolynomialFeatures(total_degree), LinearRegression())
# polyreg.fit(x, y)

# x_smooth = np.linspace(x.min(), x.max(), 300).reshape(-1, 1)
# y_smooth = polyreg.predict(x_smooth)

# plt.plot(x_smooth, y_smooth, color='gray')


# """
# max y value
# """

# max_coords = []
# for x_elem in adv_acc:
#     y_elem = quality[adv_acc == x_elem]
#     max_y = y_elem.max()
#     max_coords.append((x_elem, max_y))
#     plt.scatter(x_elem, max_y, marker='o', facecolors='none', edgecolors='gray', s=3)

# max_coords = sorted(max_coords, key=lambda x: x[0])
# max_x, max_y = zip(*max_coords)

# # approximate with a polynomial regression
# x = np.array(max_x).reshape(-1, 1)
# y = np.array(max_y)
# maxy_degree = 1
# polyreg = make_pipeline(PolynomialFeatures(maxy_degree), LinearRegression())
# polyreg.fit(x, y)
# x_smooth = np.linspace(x.min(), x.max(), 300).reshape(-1, 1)
# y_smooth = polyreg.predict(x_smooth)
# plt.plot(x_smooth, y_smooth, color='red', linestyle='--')


# print("\nAll above the max-y regression line:")
# above = [data[(data['acc_rank_delta'] == x_elem) & (data['quality'] == y_elem)].iloc[0] for x_elem, y_elem in max_coords if y_elem > polyreg.predict([[x_elem]])]
# above = pd.DataFrame(above)
# above = above.sort_values(by='final_score', ascending=False)
# display(HTML(above.to_html()))

# # print("\nMax Quality per Accuracy (sorted by x coordinate):")
# # max_coords_df = max_coords_df.sort_values(by='adv_acc')
# # display(HTML(max_coords_df.to_html()))

# max_coords_df = max_coords_df.sort_values(by='final_score', ascending=False)
# print("\nMax Quality per Accuracy (sorted by final_score):")
# display(HTML(max_coords_df.to_html()))

# """
# plot
# """

# plt.xlabel('Adversarial Accuracy (lower is better)')
# plt.ylabel('Perceptual Quality (higher is better)')
# plt.title('Trade-off between Adversarial Accuracy and Perceptual Quality')

# # legend for markers
# legend_elements = [plt.Line2D([0], [0], marker='X', color='w', label='Word', markerfacecolor='gray', markersize=10),
#                    plt.Line2D([0], [0], marker='o', color='w', label='Circle', markerfacecolor='gray', markersize=10),
#                    plt.Line2D([0], [0], marker='D', color='w', label='Diamond', markerfacecolor='gray', markersize=10),
#                    plt.Line2D([0], [0], marker='s', color='w', label='Square', markerfacecolor='gray', markersize=10),
#                    plt.Line2D([0], [0], marker='^', color='w', label='Knit', markerfacecolor='gray', markersize=10)]

# legend_elements.append(plt.Line2D([0], [0], marker='None', color='w', label=''))

# # legend for opacity
# for opacity, color in colors.items():
#     legend_elements.append(plt.Line2D([0], [0], marker='o', color='w', label=f'Opacity {opacity}', markerfacecolor=color, markersize=10))

# legend_elements.append(plt.Line2D([0], [0], marker='None', color='w', label=''))

# # legend for max y value
# legend_elements.append(plt.Line2D([0], [0], color='red', linestyle='--', label=f'Poly Regression for max quality (degree={maxy_degree})'))

# # legend for polynomial regression
# legend_elements.append(plt.Line2D([0], [0], color='gray', label=f'Poly Regression for full dataset (degree={total_degree})'))

# plt.legend(handles=legend_elements, title='Legend', loc='upper left')

# # add grid
# plt.grid(True, which='both', linestyle='--', linewidth=0.5)
# plt.minorticks_on()
# plt.gca().xaxis.set_minor_locator(AutoMinorLocator())
# plt.gca().yaxis.set_minor_locator(AutoMinorLocator())

# plt.tight_layout()
# plt.show()


,mask,opacity,acc_rank_delta,quality,final_score
4,circle,170,-344.2015,0.074388,344.275888
19,square,170,-289.6950,0.119755,289.814755
3,circle,140,-233.4175,0.123174,233.540674
9,diamond,170,-220.9270,0.118757,221.045757
18,square,140,-152.8820,0.184109,153.066109
2,circle,110,-132.1120,0.205065,132.317065
8,diamond,140,-109.5700,0.183102,109.753102
17,square,110,-70.6540,0.263996,70.917996
1,circle,80,-62.3100,0.314282,62.624282
7,diamond,110,-48.6240,0.258554,48.882554
